In [1]:
import torch
import torch.nn as nn
import torch.utils.data as tud
from tqdm.notebook import tqdm
from pprint import pprint
import matplotlib.pyplot as plt
import numpy as np
import importlib
import sys
sys.path.append("../../")
import autoregressive

In [2]:
# chess

with open("data/chess_book.txt", encoding = "utf-8") as file:
    text = file.read()
print(len(text))
print(text[:1000])

556949
﻿The Project Gutenberg EBook of Chess Strategy, by Edward Lasker

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.org/license


Title: Chess Strategy

Author: Edward Lasker

Translator: J. Du Mont

Release Date: November 11, 2012 [EBook #5614]

Language: English


*** START OF THIS PROJECT GUTENBERG EBOOK CHESS STRATEGY ***




Produced by John Mamoun <mamounjo@umdnj.edu>, Charles
Franks, and the Online Distributed Proofreaders website.








INFORMATION ABOUT THIS E-TEXT EDITION



The following is an e-text of "Chess Strategy," second edition, (1915)
by Edward Lasker, translated by J. Du Mont.

This e-text contains the 167 chess and checkers board game
diagrams appearing in the original book, all in the form of
ASCII line drawings. The following is a key to the diagrams:

For chess

In [3]:
len(text)

556949

In [4]:
vocabulary = set(text)
len(vocabulary)

92

In [5]:
char2i = {c:i for i, c in enumerate(sorted(vocabulary), 3)}
char2i["<PAD>"] = 0
char2i["<START>"] = 1
char2i["<END>"] = 2
i2char = {i:c for i, c in enumerate(sorted(vocabulary), 3)}
i2char[0] = "<PAD>"
i2char[1] = "<START>"
i2char[2] = "<END>"

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

device(type='cuda')

In [7]:
length = 17
lines = []
for i in range(len(text))[:-length]:
    lines.append(text[i:length + i])
print(len(text))
print(len(lines))
print(lines[:5])
encoded = torch.tensor([[char2i[c] for c in l] for l in tqdm(lines)]).to(device).long()
print(encoded.shape)

556949
556932
['\ufeffThe Project Gute', 'The Project Guten', 'he Project Gutenb', 'e Project Gutenbe', ' Project Gutenber']



torch.Size([556932, 17])


# fit

In [9]:
importlib.reload(autoregressive)    
net = autoregressive.TransformerEncoder(char2i, i2char)
net.to(device)
performance = net.fit(encoded, save_path = "checkpoints/autoregressive_transformer.pt", progress_bar = 0)
net.save_architecture("architectures/autoregressive_transformer.architecture")

Model: Autoregressive Transformer Encoder
Tokens in the in vocabulary: 95
Tokens in the out vocabulary: 95
Max sequence length: 16
Embedding dimension: 32
Feedforward dimension: 128
Layers: 2
Attention heads: 2
Activation: relu
Dropout: 0.0
Trainable parameters: 44,799

Epoch | Train                 | Minutes
      | Loss     | Error Rate |
---------------------------------------
    1 |   1.3970 |     39.021 |     0.8
    2 |   1.1669 |     34.161 |     1.6
    3 |   1.1244 |     33.149 |     2.4
    4 |   1.1028 |     32.608 |     3.2
    5 |   1.0887 |     32.270 |     4.0


In [10]:
# includes all the information about the epoch and the model, useful for reproducibility

performance

,epoch,train_loss,train_error_rate,minutes,learning_rate,weight_decay,model,max_sequence_length,embedding_dimension,feedforward_dimension,layers,attention_heads,activation,dropout,parameters
0,1,1.396962,39.021460,0.780697,0.001,0,Autoregressive Transformer Encoder,16,32,128,2,2,relu,0.0,44799
1,2,1.166885,34.161172,1.583761,0.001,0,Autoregressive Transformer Encoder,16,32,128,2,2,relu,0.0,44799
2,3,1.124363,33.148661,2.385385,0.001,0,Autoregressive Transformer Encoder,16,32,128,2,2,relu,0.0,44799
3,4,1.102792,32.608492,3.189483,0.001,0,Autoregressive Transformer Encoder,16,32,128,2,2,relu,0.0,44799
4,5,1.088718,32.269503,4.013829,0.001,0,Autoregressive Transformer Encoder,16,32,128,2,2,relu,0.0,44799


In [11]:
# the input for testing

net.tensor2text(encoded[:5])

['\ufeffThe Project Gute',
 'The Project Guten',
 'he Project Gutenb',
 'e Project Gutenbe',
 ' Project Gutenber']

# predict

In [12]:
importlib.reload(autoregressive)    
net = autoregressive.load_architecture("architectures/autoregressive_transformer.architecture")
net.load_state_dict(torch.load("checkpoints/autoregressive_transformer.pt"))
net.to(device)

idx, log_probabilities = net.predict(encoded[:5], main_progress_bar = False, progress_bar = 0)

net.tensor2text(idx)

Model: Autoregressive Transformer Encoder
Tokens in the in vocabulary: 95
Tokens in the out vocabulary: 95
Max sequence length: 16
Embedding dimension: 32
Feedforward dimension: 128
Layers: 2
Attention heads: 2
Activation: relu
Dropout: 0.0
Trainable parameters: 44,799



../../autoregressive.py:356: RuntimeWarning: Max sequence length exceded, only using the last 16 tokens of the input. You can disable this warning with the warn_last_tokens parameter of the forward method.
  warnings.warn(f"Max sequence length exceded, only using the last {self.architecture['max_sequence_length']} tokens of the input. You can disable this warning with the warn_last_tokens parameter of the forward method.", category = RuntimeWarning)


['\ufeffThe Project Gutenberg-tm order to th',
 'The Project Gutenberg-tm order to the',
 'he Project Gutenberg-tm order to the ',
 'e Project Gutenberg-tm order to play ',
 ' Project Gutenberg-tm order to play P']

# greedy_search

In [13]:
importlib.reload(autoregressive)    
net = autoregressive.load_architecture("architectures/autoregressive_transformer.architecture")
net.load_state_dict(torch.load("checkpoints/autoregressive_transformer.pt"))
net.to(device)
indexes, log_probabilities = net.greedy_search(encoded[:5], progress_bar = False)

print(log_probabilities)
net.tensor2text(indexes)

Model: Autoregressive Transformer Encoder
Tokens in the in vocabulary: 95
Tokens in the out vocabulary: 95
Max sequence length: 16
Embedding dimension: 32
Feedforward dimension: 128
Layers: 2
Attention heads: 2
Activation: relu
Dropout: 0.0
Trainable parameters: 44,799

tensor([ -9.3253,  -9.6260,  -9.7838, -11.7067, -12.6741], device='cuda:0')


['\ufeffThe Project Gutenberg-tm order to th',
 'The Project Gutenberg-tm order to the',
 'he Project Gutenberg-tm order to the ',
 'e Project Gutenberg-tm order to the K',
 ' Project Gutenberg-tm order to the Ki']

# sample

In [14]:
importlib.reload(autoregressive)    
net = autoregressive.load_architecture("architectures/autoregressive_transformer.architecture")
net.load_state_dict(torch.load("checkpoints/autoregressive_transformer.pt"))
net.to(device)
indexes, log_probabilities = net.sample(encoded[:5], progress_bar = False)

print(log_probabilities)
net.tensor2text(indexes)

Model: Autoregressive Transformer Encoder
Tokens in the in vocabulary: 95
Tokens in the out vocabulary: 95
Max sequence length: 16
Embedding dimension: 32
Feedforward dimension: 128
Layers: 2
Attention heads: 2
Activation: relu
Dropout: 0.0
Trainable parameters: 44,799

tensor([-20.0712, -21.3544, -22.5175, -30.8438, -13.7993], device='cuda:0')


['\ufeffThe Project Gutenberg-tm a shilst.\n\n',
 'The Project Gutenberg-tm c5. K-Kt2   ',
 'he Project Gutenberg-tmment.\n\nA is ar',
 'e Project Gutenberg-tm ouf very wwins',
 ' Project Gutenberg-tm the weakness th']

# beam_search

In [15]:
importlib.reload(autoregressive)    
net = autoregressive.load_architecture("architectures/autoregressive_transformer.architecture")
net.load_state_dict(torch.load("checkpoints/autoregressive_transformer.pt"))
net.to(device)
indexes, log_probabilities = net.beam_search(encoded[:5], progress_bar = 0)

print(log_probabilities)
pprint([net.tensor2text(t) for t in indexes])

Model: Autoregressive Transformer Encoder
Tokens in the in vocabulary: 95
Tokens in the out vocabulary: 95
Max sequence length: 16
Embedding dimension: 32
Feedforward dimension: 128
Layers: 2
Attention heads: 2
Activation: relu
Dropout: 0.0
Trainable parameters: 44,799

tensor([[ -9.3253, -10.2871, -10.6251, -10.6906, -10.7245],
        [ -9.6260, -10.7373, -10.9493, -11.0146, -11.1352],
        [ -9.7838, -10.7810, -11.0273, -11.1860, -11.8575],
        [-11.0765, -11.2367, -11.7067, -12.1978, -12.2495],
        [-12.1441, -12.2229, -12.6741, -12.6960, -13.1410]], device='cuda:0')
[['\ufeffThe Project Gutenberg-tm order to th',
  '\ufeffThe Project Gutenberg-tm the pawns o',
  '\ufeffThe Project Gutenberg-tm the pawns i',
  '\ufeffThe Project Gutenberg-tm the pawns t',
  '\ufeffThe Project Gutenberg-tm order to pl'],
 ['The Project Gutenberg-tm order to the',
  'The Project Gutenberg-tm order to pla',
  'The Project Gutenberg-tm the pawns th',
  'The Project Gutenberg-tm the pawns in'